In [1]:
import numpy as np

In [3]:
x = np.load('tinyin_train.npy')
y = np.load('tinyin_train_label.npy')

In [12]:
x_test = np.load('tinyin_test.npy')

In [13]:
x_test[0].shape

(64, 64, 3)

In [9]:
xx = np.load('train_origin.npy')

In [11]:
xx[0].shape

(32, 32)

In [17]:
from PIL import Image
import numpy as np

def convert_to_grayscale(image_path):
    # 打开彩色图像
    img = Image.open(image_path)
    # 将图像调整为 (64, 64) 的尺寸
    img = img.resize((64, 64))
    # 将图像转换为灰度
    grayscale_img = img.convert("L")
    # 转换为 (64, 64) 的 numpy 矩阵
    grayscale_matrix = np.array(grayscale_img)
    
    return grayscale_matrix

'''
# 示例用法
image_path = 'n01443537_0.JPEG'  # 替换为你的图像路径
gray_matrix = convert_to_grayscale(image_path)
print(gray_matrix.shape)  # 应为 (64, 64)
'''

(64, 64)


In [23]:
import os
from PIL import Image
import numpy as np

def load_train_images_and_labels(train_folder_path):
    images = []
    labels = []
    
    # 获取 train 文件夹下的所有类别子文件夹
    class_folders = sorted([d for d in os.listdir(train_folder_path) if os.path.isdir(os.path.join(train_folder_path, d))])
    
    for label, class_folder in enumerate(class_folders):
        class_path = os.path.join(train_folder_path, class_folder, 'images')
        
        # 检查 images 文件夹是否存在
        if not os.path.exists(class_path):
            continue
        
        # 获取每个类别文件夹内的所有JPEG文件
        for file_name in os.listdir(class_path):
            if file_name.endswith('.JPEG') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
                file_path = os.path.join(class_path, file_name)
                
                # 打开并转换为 (64, 64) 灰度图像
                img = Image.open(file_path).convert("L").resize((64, 64))
                img_array = np.array(img)
                
                # 保存图像数据和对应的标签
                images.append(img_array)
                labels.append(label)
    
    # 转换为 NumPy 数组
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# 使用示例
train_folder_path = 'tiny-imagenet-200/train'  # 替换为你的train文件夹路径
images, labels = load_train_images_and_labels(train_folder_path)
print(f'Images shape: {images.shape}, Labels shape: {labels.shape}')


Images shape: (100000, 64, 64), Labels shape: (100000,)


In [26]:
import numpy as np
from collections import defaultdict

def sample_and_remove_data(images, labels, num_samples=50):
    sampled_images = []
    sampled_labels = []
    indices_to_remove = []

    # 获取每个标签的索引列表
    label_indices = defaultdict(list)
    for idx, label in enumerate(labels):
        label_indices[label].append(idx)

    # 从每个标签的索引中随机选取 num_samples 个
    for label, indices in label_indices.items():
        if len(indices) < num_samples:
            raise ValueError(f"标签 {label} 的样本数量不足 {num_samples} 个。")
        
        selected_indices = np.random.choice(indices, num_samples, replace=False)
        
        # 添加选取的数据到采样结果中
        sampled_images.extend(images[selected_indices])
        sampled_labels.extend(labels[selected_indices])
        
        # 将这些索引加入要删除的列表
        indices_to_remove.extend(selected_indices)

    # 转换为 NumPy 数组
    sampled_images = np.array(sampled_images)
    sampled_labels = np.array(sampled_labels)

    # 从原数据集中删除已选数据
    images = np.delete(images, indices_to_remove, axis=0)
    labels = np.delete(labels, indices_to_remove, axis=0)

    return sampled_images, sampled_labels, images, labels

# 使用示例
sampled_images, sampled_labels, remaining_images, remaining_labels = sample_and_remove_data(images, labels, num_samples=50)
print(f'Sampled Images shape: {sampled_images.shape}, Sampled Labels shape: {sampled_labels.shape}')
print(f'Remaining Images shape: {remaining_images.shape}, Remaining Labels shape: {remaining_labels.shape}')


Sampled Images shape: (10000, 64, 64), Sampled Labels shape: (10000,)
Remaining Images shape: (90000, 64, 64), Remaining Labels shape: (90000,)


In [27]:
np.save('tiny_origin_train.npy', remaining_images)
np.save('tiny_origin_train_label.npy', remaining_labels)
np.save('tiny_origin_test.npy', sampled_images)
np.save('tiny_origin_test_label.npy', sampled_labels)